In [ ]:
print(f'lolmax')

# Evaluating Linear Probes on the Model Output and Action

In [ ]:
import sys
import argparse
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch as t
import importlib
import os
import tqdm
import pickle
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pandas import DataFrame, json_normalize
from tqdm import tqdm
import os
import numpy as np
from typing import Dict, Any, List, Tuple
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score

In [ ]:
sys.path.append('../linear-probes/')
from probes import LinearProbe
sys.path.append(os.path.dirname(os.path.abspath('.')))

from evaluate_utils import evaluate_probe_on_activation_dataset
from plots import plot_behavior_distribution, plot_roc_curves, add_roc_curves, print_metrics, plot_roc_curve_eval
import probes
from pprint import pprint as pp

from probe_datasets import AmongUsDataset, TruthfulQADataset, DishonestQADataset, RepEngDataset, RolePlayingDataset, ApolloProbeDataset
from configs import config_phi4, config_gpt2, config_llama3
base_config = config_phi4
amongus_expt_name: str = "2025-02-01_phi_phi_100_games_v3"
# layers_to_work_on: List[int] = list(range(base_config["num_layers"]))
layers_to_work_on: List[int] = [20]

In [ ]:
from configs import config_phi4, config_gpt2, config_llama3
base_config = config_phi4

In [ ]:
layer: int = layers_to_work_on[0]
probe_dataset_name: str = 'RepEngDataset'

probe = LinearProbe(base_config["activation_size"])
        
checkpoint_path = f'../linear-probes/checkpoints/{probe_dataset_name}_probe_{base_config["short_name"]}_layer{layer}.pkl'

with open(checkpoint_path, 'rb') as f:
    probe.model = pickle.load(f).model
    print(f'Loaded probe trained on {probe_dataset_name} for layer {layer}.')

config = base_config.copy()
config["layer"] = str(layer)

In [ ]:
dataset = AmongUsDataset(config, model=None, tokenizer=None, device='cpu', expt_name=config['expt_name'], test_split=1)
all_probe_outputs = []
chunk_size: int = 500
list_of_chunks_to_eval = [0, 1]
row_indices = []
all_token_scores = []

In [ ]:
for chunk_idx in tqdm(list_of_chunks_to_eval):
    test_acts_chunk = dataset.get_test_acts(chunk_idx)
    
    # Store the row indices for this chunk
    start_idx = chunk_idx * chunk_size
    end_idx = start_idx + len(test_acts_chunk)
    row_indices.extend(range(start_idx, end_idx))
    
    chunk_probe_outputs, _, token_scores = evaluate_probe_on_activation_dataset(
        chunk_data=test_acts_chunk,
        probe=probe,
        device=None,
        num_tokens=None,
        verbose=False,
        output_token_scores=True,
    )
    all_probe_outputs.extend(chunk_probe_outputs)
    all_token_scores.extend(token_scores)

In [ ]:
from transformers import AutoTokenizer

model_name = config['model_name']
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
len(all_token_scores), len(chunk_probe_outputs)

In [ ]:
len(all_token_scores)

In [ ]:
len(all_token_scores[0])

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML

In [ ]:
dataset.agent_logs_df.iloc[row_indices[33]]

In [ ]:
row_index = 0

row_token_scores = all_token_scores[row_index]
row_token_scores = np.array(row_token_scores)
raw_df = dataset.agent_logs_df
actual_row_idx = row_indices[row_index]

fp = dataset.agent_logs_row_to_full_prompt(raw_df.iloc[actual_row_idx])
encoded = tokenizer.encode(fp, return_tensors="pt")

full_prompt_tokens = []

for token_id in encoded[0]:
    token = tokenizer.decode(token_id)
    full_prompt_tokens.append(token)

words = full_prompt_tokens

In [ ]:
all_index_words = []
all_index_scores = []
all_index_label = []
all_index_action = []

for row_index in tqdm(range(len(all_token_scores))):
    row_token_scores = all_token_scores[row_index]
    row_token_scores = np.array(row_token_scores)
    actual_row_idx = row_indices[row_index]
    raw_df = dataset.agent_logs_df
    fp = dataset.agent_logs_row_to_full_prompt(raw_df.iloc[actual_row_idx])
    identity = raw_df.iloc[actual_row_idx]['player.identity']
    action = raw_df.iloc[actual_row_idx]['action']
    encoded = tokenizer.encode(fp, return_tensors="pt")

    full_prompt_tokens = []

    for token_id in encoded[0]:
        token = tokenizer.decode(token_id)
        full_prompt_tokens.append(token)

    row_words = full_prompt_tokens
    all_index_words.append(row_words)
    all_index_scores.append(row_token_scores)
    all_index_label.append(identity)
    all_index_action.append(action)

In [ ]:
df = pd.DataFrame(
    {
        'string_tokens': all_index_words, 
        'token_scores': all_index_scores, 
        'label': all_index_label,
        'action': all_index_action
    }
)
system_identifier = ['<|im_start|>', 'system', '<|im_sep|>']
user_identifier = ['<|im_start|>', 'user', '<|im_sep|>']
model_output_identifier = ['<|im_start|>', 'assistant', '<|im_sep|>']
model_action_identifier = ['[', 'Action']
model_action_identifier_2 = ['[', 'action']

def find_sublist_start_idx(tokens, sublist, reversed=False):
    n = len(tokens)
    m = len(sublist)
    
    if reversed:
        # Search from the end to find the last occurrence
        for i in range(n - m, -1, -1):
            if tokens[i:i+m] == sublist:
                return i
    else:
        # Original behavior - find first occurrence
        for i in range(n - m + 1):
            if tokens[i:i+m] == sublist:
                return i
    return 0

In [ ]:
# Define a soft mix parameter between mean and max aggregation
k = 0  # 0 = pure mean, 1 = pure max

df['system_start_idx'] = df['string_tokens'].apply(lambda tokens: find_sublist_start_idx(tokens, system_identifier))
df['user_start_idx'] = df['string_tokens'].apply(lambda tokens: find_sublist_start_idx(tokens, user_identifier))
df['model_output_start_idx'] = df['string_tokens'].apply(lambda tokens: find_sublist_start_idx(tokens, model_output_identifier))
df['model_action_start_idx'] = df['string_tokens'].apply(lambda tokens: \
    max(find_sublist_start_idx(tokens, model_action_identifier, reversed=True), \
        find_sublist_start_idx(tokens, model_action_identifier_2, reversed=True)))

# Define a function that creates a soft mix between mean and max
def soft_agg(values):
    if len(values) == 0:
        return 0
    return k * np.max(values) + (1-k) * np.mean(values)

df['system_scores'] = df.apply(lambda row: soft_agg(row['token_scores'][row['system_start_idx']:row['user_start_idx']]), axis=1)
df['user_scores'] = df.apply(lambda row: soft_agg(row['token_scores'][row['user_start_idx']:row['model_output_start_idx']]), axis=1)
df['model_output_scores'] = df.apply(lambda row: soft_agg(row['token_scores'][row['model_output_start_idx']:]), axis=1)
df['model_action_scores'] = df.apply(lambda row: soft_agg(row['token_scores'][row['model_action_start_idx']:]), axis=1)

df['ground_truth'] = df['label'].apply(lambda x: 1 if x == 'Crewmate' else 0)

In [ ]:
df.shape, df.columns

In [ ]:
df.head(3)

In [ ]:
def plot_roc_curves(df):
    prediction_cols = ['system_scores', 'user_scores', 'model_output_scores', 'model_action_scores']
    legend_names = ['+System', '+User', '+Thinking', 'Action']
    
    # Create figure with square shape
    fig = go.Figure(layout=go.Layout(
        width=500, 
        height=500,
        font=dict(family="Computer Modern", size=14),  # LaTeX-style font
        template="plotly_white"
    ))
    
    # Color palette for elegant, deep colors
    colors = ['#1f77b4', '#d62728', '#2ca02c', '#9467bd']
    
    # Add ROC curves with thicker lines and calculate AUROC
    for i, col in enumerate(prediction_cols):
        fpr, tpr, thresholds = roc_curve(df['ground_truth'], df[col])
        auc_score = roc_auc_score(df['ground_truth'], df[col])
        
        # Format name with AUROC value
        name = f"{col} (AUC = {auc_score:.3f})"
        
        fig.add_trace(go.Scatter(
            x=fpr, 
            y=tpr, 
            mode='lines', 
            name=f'{legend_names[i]} (AUC = {auc_score:.3f})',
            line=dict(color=colors[i], width=3),
        ))
    
    # Add random baseline (y=x) as dashed black line
    fig.add_trace(go.Scatter(
        x=[0, 1], 
        y=[0, 1], 
        mode='lines', 
        name='Random (AUC = 0.500)',
        line=dict(color='black', width=2, dash='dash'),
    ))
    
    # Update layout for research paper quality
    fig.update_layout(
        xaxis=dict(
            title='False Positive Rate',
            titlefont=dict(size=16),
            tickfont=dict(size=14),
            gridcolor='#E9E9E9',
        ),
        yaxis=dict(
            title='True Positive Rate',
            titlefont=dict(size=16),
            tickfont=dict(size=14),
            gridcolor='#E9E9E9',
        ),
        legend=dict(
            x=0.45,
            y=0.1,
            bgcolor='rgba(255, 255, 255, 0.8)',
            bordercolor='#E2E2E2',
            borderwidth=1,
            traceorder='reversed',  # Reverse the order of the legend items
        ),
        plot_bgcolor='white',
        margin=dict(l=60, r=30, t=30, b=60),
    )
    
    fig.update_layout(width=520, height=500)

    return fig

In [ ]:
fig = plot_roc_curves(df)
fig.show()

In [ ]:
# save as a high quality pdf
fig.write_image("plots/output_action_roc_mean.pdf", width=500, height=500)

## Filter on SPEAK Actions

In [ ]:
df_speak = df[df['action'].str.contains('SPEAK')]

In [ ]:
fig = plot_roc_curves(df_speak)
fig.show()

In [ ]:
fig.write_image("plots/output_action_roc_mean_speak.pdf", width=500, height=500)

In [ ]:
# Train probes on just action / speech actions and thinking + actions (model output)